In [20]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)
import model
import detect_utils
import cv2
import json
import matplotlib.pyplot as plt



In [21]:

print("Torch version:", torch.__version__)

# create own Dataset
training_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.train_coco, transforms=get_transform()
)

# training DataLoader
training_loader = torch.utils.data.DataLoader(
    training_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in training_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = model.get_model(device = device, model_name='v2')


# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(training_loader)


Torch version: 1.13.1+cpu
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
[{'boxes': tensor([]), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([1199]), 'area': tensor([]), 'iscrowd': tensor([], dtype=torch.int64)}]


ValueError: Caught ValueError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "C:\Users\thoma\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\thoma\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\thoma\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\thoma\Documents\Github\Foocus-Logodetection\utils.py", line 52, in __getitem__
    labels = torch.as_tensor(labels, dtype=torch.int64)
ValueError: too many dimensions 'str'


In [ ]:

# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    
    for imgs, annotations in training_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        print("imgs:", imgs)
        print("num: ")
        print(annotations[0]['boxes'].size() == torch.Size([0]))
        
        #if the annotations are emptpy, skip
        if annotations[0]['boxes'].size() == torch.Size([0]):
            continue
        
        print("annotations:", annotations)
        # pass if there are no annotations
        # if len(annotations) == 0:
        #     continue
        # if 
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

In [ ]:
#load json file

# with open(config.test_coco) as f:
#     json_data = json.load(f)



num_correct = 0
total = 0
# create test Dataset
testing_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.test_coco, transforms=get_transform()
)
#testing DataLoader
testing_loader = torch.utils.data.DataLoader(
    testing_dataset,
    batch_size=config.test_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


model.eval()

# Evaluate the model
with torch.no_grad():
    
    for imgs, annotations in testing_loader:
        print("len")
        print(len(imgs))
        print(len(annotations))
        print("____")

        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        print("annotations: " + str(annotations[0]['boxes']))
        print("annotation labels: " + str(annotations[0]['labels']))
        annotationIndex = annotations[0]['labels'].item()
        print("annotationIndex: " + str(annotationIndex))

        
        # if annotations[0]['boxes'].size() == torch.Size([0]):
        #     continue
        # boxes, classes, labels = detect_utils.predict(image, model, device, 0.8)

        # image = detect_utils.draw_boxes(boxes, classes, labels, image)

        # i = 0
        
        for image in imgs:
            # annotaion = annotations[i]
            # i += 1

            # image = image.transpose(0, 1)
            image = image.transpose(0, 2)
            # puts the color in the right place but the image must be rotated 90 degrees
            image = image.transpose(0, 1)

            image = image.numpy()

            boxes, pred_classes, labels = detect_utils.predict(image, model, device, 0.8)

            # print(annotaion)
            print("boxes: ")
            print(boxes)
            print("pred_classes:")
            print(pred_classes)
            print("labels: ")
            print(labels)


            image = detect_utils.draw_boxes(boxes, pred_classes, labels, image)
            
            plt.imshow(image) 
            plt.show()


        
        # cv2.waitKey(0)
            
        # predictions = model(data)
        # num_correct += (predictions == labels).sum()
        # total += labels.size(0)

    # print(f"Test Accuracy of the model: {float(num_correct)/float(total)*100:.2f}")